
# Asset Report — Wind 1 (ERCOT)
This notebook executes **all core steps** from the project brief for one asset (Wind 1 in ERCOT), and presents the results clearly with tables and plots.

**Scope covered:**
- Expected generation (by month, Peak/Off-Peak)
- Four 5-year **fixed prices** ($/MWh): RT/DA × Hub/Node
- **Price breakdown** (Hub capture, Basis, DA–RT spread, Negative-price clause, Risk add-on)
- Risk target **P75** (and P50 / P90 sensitivity)
- Discussion prompts: volume & price risk, negative price impact


In [ ]:

import yaml
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

from src.data.loader import load_data
from src.features.calendar import make_calendar_index
from src.models.shape import hub_hourly_shape
from src.models.price import (
    shape_monthly_to_hourly,
    basis_stats,
    da_spread_stats,
    simulate_prices
)
from src.models.volume import bootstrap_generation
from src.pricing.solve_price import (
    solve_product_prices,
    expected_generation_monthly,
    compute_price_breakdown,
    solve_prices_for_levels,
)

CONFIG_PATH = "configs/project.yaml"
ASSET_NAME = "Wind_A"
MARKET = "ERCOT"
P_LEVEL = 0.75
P_GRID = [0.50, 0.75, 0.90]
SEED = 42
SCENARIOS = 1000
np.set_printoptions(suppress=True, linewidth=120)

cfg = yaml.safe_load(open(CONFIG_PATH, "r"))
start_year = cfg["start_year"]
end_year = cfg["end_year"]
neg_rule = cfg.get("negative_price_rule", "include")


In [ ]:

hist_all, fwd_all = load_data(cfg)
hist = hist_all[hist_all["market"] == MARKET].copy()
hist_asset = hist_all[(hist_all["market"] == MARKET) & (hist_all["asset"] == ASSET_NAME)].copy()

print("Historical rows (market):", len(hist))
print("Historical rows (asset):", len(hist_asset))
display(hist_asset.head(3))

num_cols = ["gen_mwh","rt_hub","rt_node","da_hub","da_node"]
print("Numeric dtype check (asset):")
display(hist_asset[num_cols].dtypes)


In [ ]:

cal = make_calendar_index(start_year, end_year)
cal.head()


In [ ]:

shape_tbl = hub_hourly_shape(hist)
bs_rt = basis_stats(hist, price_col_hub="rt_hub", price_col_node="rt_node")
spr_hub = da_spread_stats(hist, level="hub")
spr_node = da_spread_stats(hist, level="node")

print("Shape rows:", len(shape_tbl), "| Basis rows:", len(bs_rt), "| Spread rows:", len(spr_hub), len(spr_node))


In [ ]:

hh_all = shape_monthly_to_hourly(fwd_all, shape_tbl, cal)
hh = hh_all[hh_all["market"] == MARKET].copy().sort_values("ts").reset_index(drop=True)
print("Hourly hub forward rows (ERCOT):", len(hh))
display(hh.head(5))


In [ ]:

sims_all = simulate_prices(hh, bs_rt, spr_hub, spr_node, n_scenarios=SCENARIOS, seed=SEED)
sims = sims_all[sims_all["market"] == MARKET].copy()
display(sims.head(3))


In [ ]:

vol_tbl = (
    hist_all.assign(month=hist_all["date"].dt.to_period("M").dt.to_timestamp(), hour=hist_all["he"])
            .groupby(["asset","market","month","hour"])["gen_mwh"]
            .apply(list).reset_index(name="samples")
)

rng = np.random.default_rng(SEED)
gen_all = bootstrap_generation(cal, vol_tbl, n_scenarios=SCENARIOS, rng=rng)
gen = gen_all[(gen_all["asset"] == ASSET_NAME) & (gen_all["market"] == MARKET)].copy()

display(gen.head(3))
print("Generation scenarios rows (asset):", len(gen))


In [ ]:

eg = expected_generation_monthly(gen)
eg_asset = eg[eg["asset"] == ASSET_NAME].copy()
display(eg_asset.head(6))

Path("results").mkdir(exist_ok=True)
eg_asset.to_csv("results/expected_generation_wind1.csv", index=False)
print("Saved results/expected_generation_wind1.csv")


In [ ]:

products = ["RT_HUB","RT_NODE","DA_HUB","DA_NODE"]

sims_mkt = sims.copy()

prices_p75 = solve_product_prices(sims_mkt, gen, products, p_level=P_LEVEL, negative_rule=neg_rule)
prices_p75 = prices_p75[prices_p75["asset"] == ASSET_NAME].copy().reset_index(drop=True)
display(prices_p75)

prices_grid = solve_prices_for_levels(sims_mkt, gen, products, P_GRID, negative_rule=neg_rule)
prices_grid = prices_grid[prices_grid["asset"] == ASSET_NAME].copy().reset_index(drop=True)
display(prices_grid.head(8))

prices_p75.to_csv("results/fixed_prices_wind1_p75.csv", index=False)
prices_grid.to_csv("results/fixed_prices_wind1_pgrid.csv", index=False)
print("Saved fixed price results for Wind 1.")


In [ ]:

breakdown_all = compute_price_breakdown(
    hourly_hub_fwd=hh,
    basis_mean_rt=bs_rt[["market","month","hour","mean"]],
    da_spr_mean_hub=spr_hub[["market","month","hour","mean"]],
    da_spr_mean_node=spr_node[["market","month","hour","mean"]],
    sims=sims,
    gen_df=gen,
    products=products,
    p_level=P_LEVEL
)
bk = breakdown_all[breakdown_all["asset"] == ASSET_NAME].copy().reset_index(drop=True)
display(bk)

bk.to_csv("results/price_breakdown_wind1.csv", index=False)
print("Saved results/price_breakdown_wind1.csv")


In [ ]:

plt.figure()
x = np.arange(len(products))
vals = [float(prices_p75[prices_p75['product']==p]['fixed_price'].values[0]) for p in products]
plt.bar(x, vals)
plt.xticks(x, products, rotation=0)
plt.ylabel("$/MWh")
plt.title("Wind 1 — Fixed Prices (P75)")
plt.tight_layout()
plt.show()


In [ ]:

prod = "DA_NODE"
row = bk[bk["product"] == prod].iloc[0].to_dict()

steps = ["A_hub_capture","B_basis","C_da_rt","D_neg_rule","E_risk"]
values = [row[s] for s in steps]
labels = ["A: Hub","B: Basis","C: DA-RT","D: Neg Rule","E: Risk"]

plt.figure()
plt.bar(range(len(values)), values)
plt.xticks(range(len(values)), labels, rotation=0)
plt.ylabel("$/MWh")
plt.title(f"Wind 1 — Price Breakdown Waterfall ({prod})\nSum = P* (include): {row['P_star_include']:.2f},  P* (zero): {row['P_star_zero']:.2f}")
plt.tight_layout()
plt.show()


In [ ]:

plt.figure()
for p in products:
    sub = prices_grid[prices_grid["product"] == p]
    xs = sub["p_level"].values
    ys = sub["fixed_price"].values
    plt.plot(xs, ys, marker="o", label=p)
plt.xlabel("P-level target")
plt.ylabel("Fixed price ($/MWh)")
plt.title("Wind 1 — Fixed Price vs Risk Appetite")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:

summary = sims.groupby("ts")[["hub_rt","node_rt"]].mean().reset_index()
first = summary.iloc[:24*60]
plt.figure()
plt.plot(first["ts"], first["hub_rt"], label="Hub RT (mean)")
plt.plot(first["ts"], first["node_rt"], label="Node RT (mean)")
plt.title("Wind 1 Market (ERCOT) — First 60 Days RT Mean")
plt.xlabel("Time")
plt.ylabel("$/MWh")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



## Interpretation & Recommendations (for Wind 1)

**Volume & Price Risk** — Volume risk from bootstrap, price risk from shaped hub, basis, and DA–RT spread. The **E_risk** term quantifies the premium to meet the P75 risk appetite.

**Negative Prices** — `D_neg_rule` quantifies the uplift to exclude negatives. A larger `D` suggests more exposure to negative hours.

**Product choice** — Compare RT/DA × Hub/Node. If B (basis) is materially negative (hub>node), node-settled prices will be lower; hub-settled leaves basis risk but may price higher. DA products incorporate expected DA–RT spread.

**Stay merchant?** — Contrast expected merchant distribution against P-grid fixed prices. If expected merchant < P75 fixed price and tails are concerning, recontracting at P75 looks prudent; otherwise, staying merchant may be defendable with active risk monitoring.
